In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key = "KEY", model_name = "gpt-3.5-turbo", temperature = 0.5)

In [6]:
from dotenv import load_dotenv

load_dotenv()

False

In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    
}

In [9]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
Create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [11]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)

In [12]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity if the quiz is not at per with the cognitive and analytical abilities of the students.\
Uodate the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.

Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt = PromptTemplate(input_variables = ["subject", "quiz"], template = TEMPLATE)

In [14]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [15]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"],
                                          output_variables = ["quiz", "review"], verbose = True)

In [17]:
file_path = "C:\\Users\\Admin\\Desktop\\TEST\\data.txt"

In [18]:
file_path

'C:\\Users\\Admin\\Desktop\\TEST\\data.txt'

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [20]:
print(TEXT)

Artificial intelligence

As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). 
In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. 
They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.
Probabilistic reasoning was also employed, especially in automated medical diagnosis.
However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. 
Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.
By 1980, expert systems had come to dominate AI, and statistics was out of favor.
Work on symbolic/knowledge-based learning did continue within AI, leading to inductive logic p

In [21]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "simple"

In [23]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

c:\Users\Admin\Desktop\TEST\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Artificial intelligence

As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). 
In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. 
They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.
Probabilistic reasoning was also employed, especially in automated medical diagnosis.
However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. 
Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.
By 1980, expert systems had come to dominate AI, and statistics was

In [24]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 3880
Prompt Tokens: 3148
Completion Tokens: 732
Total Cost: 0.0061860000000000005


In [25]:
quiz = response.get("quiz")

In [26]:
quiz = json.loads(quiz)

In [27]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        {
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        }
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [28]:
quiz_table_data

[{'MCQ': 'What was the main goal of machine learning in the 1990s?',
  'Choices': 'a: Achieving artificial intelligence | b: Tackling solvable problems of a practical nature | d: Becoming a subfield of data mining | c: Focusing on symbolic approaches inherited from AI',
  'Correct': 'b'},
 {'MCQ': 'Which method can be used for optimal data compression?',
  'Choices': 'a: Linear regression | d: Decision trees | b: Logistic regression | c: Arithmetic coding',
  'Correct': 'c'},
 {'MCQ': 'What is the main focus of data mining?',
  'Choices': 'b: Discovery of unknown properties in the data | a: Prediction based on known properties | d: Evaluation of unsupervised learning methods | c: Preprocessing step to improve learner accuracy',
  'Correct': 'b'},
 {'MCQ': 'Which algorithm can be utilized for data compression by grouping similar data points into clusters?',
  'Choices': 'a: Linear regression | b: K-means clustering | d: Random Forest | c: Support Vector Machines',
  'Correct': 'b'},
 {'

In [29]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What was the main goal of machine learning in ...,a: Achieving artificial intelligence | b: Tack...,b
1,Which method can be used for optimal data comp...,a: Linear regression | d: Decision trees | b: ...,c
2,What is the main focus of data mining?,b: Discovery of unknown properties in the data...,b
3,Which algorithm can be utilized for data compr...,a: Linear regression | b: K-means clustering |...,b
4,What do loss functions express in machine lear...,d: The computational complexity of the model |...,a


In [30]:
pd.DataFrame(quiz_table_data).to_csv("machinelearning.csv", index = False)

In [31]:
from datetime import datetime
datetime.now().strftime("%d_%m_%y_%H_%M_%S")


'29_04_24_12_40_41'